In [1]:
%matplotlib ipympl
import numpy as np
import sys

# setting path for mask_utils package
sys.path.append('..')

from mask_utils.code_utils import next_prime, ura_mura
from mask_utils.imaging_utils import decode, decode_var, eff_area_vs_off_axis, solid_angle
from mask_utils.image_utils import upscale, fshift, ferosion, apply_vignetting
from mask_utils.fits_utils import read_mask_bulk
from scipy.signal import convolve
from scipy.ndimage import gaussian_filter as gaussian_filter

import matplotlib.pyplot as plt
from matplotlib import cm

## Defining the Detector PSF

In [2]:
def wfm_psfy(y):
    center, alpha, beta, sigma = 0, 0.5459735904725987, 0.7363355668833482, 0.17749677955602094 #good for both cameras on ScoX-1
    psf = 1 / np.cosh(np.abs(( y - center) / alpha) ** beta) 
    psf = gaussian_filter(psf, sigma, mode='constant', cval=0)
    
    return psf/np.sum(psf)

## Defining the Source Modeler

In [3]:
def model_source(xsh, ysh, fluence):
    _shifted = fshift(mask, xsh, ysh)
    _vignetted = apply_vignetting(_shifted, xsh, ysh, FOCAL, ELXDIM, ELYDIM, MTHICK)

    if datatype == "reconstructed":
        _detimage = convolve(_vignetted, psf, mode="same") * bulk
        _detimage_norm = _detimage/np.sum(_detimage) * fluence 
    else:
        _detimage = _vignetted * bulk
        _detimage_norm = _detimage/np.sum(_detimage) * fluence 
    
    return(_detimage_norm)

## Reading mask and detector info

In [4]:
mask_file = "F:/CodedMasks/mask_050_1040x17/mask_050_1040x17_20250710.fits"

In [5]:
mask, hdmask = read_mask_bulk(mask_file, 'MASK', header_out=True, verbose=False)
rmatrix = read_mask_bulk(mask_file, 'RMATRIX', header_out=False, verbose=False)
bulk = read_mask_bulk(mask_file, 'SENS', header_out=False, verbose=False)
bulk[bulk < 1] = 0

In [6]:
#Gets mask information and define X, Y vectors (centers and edges)
ELXDIM = hdmask['ELXDIM']
ELYDIM = hdmask['ELYDIM']
MTHICK = hdmask['MTHICK']
ELXN   = hdmask['ELXN'] 
ELYN   = hdmask['ELYN']
OPENFR=hdmask['OPENFR']
FOCAL = 202.9 + 0.150/2

In [7]:
#Upscaling mask, rmatrix, bulk
up_f_x, up_f_y = 5, 1

mask = upscale(mask, up_f_y, up_f_x)
rmatrix = upscale(rmatrix, up_f_y, up_f_x)
bulk = upscale(bulk, up_f_y, up_f_x)

ELXDIM /= up_f_x
ELYDIM /= up_f_y
ELXN   *= up_f_x
ELYN   *= up_f_y

## Generating Y PSF

In [8]:
yspan = int(15.5/ELYDIM)
ypsf = np.arange(-yspan, yspan+1) * ELYDIM
psf =  wfm_psfy(ypsf).reshape(len(ypsf), -1).T

## Source and background countrates

In [9]:
Crab_on_axis_cm2 = 2.5#2.6268
CXB_cm2 = 6.7
exposure = 1000.0

## Source off-axis angles

In [10]:
ThetaX = -15.0#19.98#-36.65#0#12.198304614766784
ThetaY = 0.0#11.97#20.9411857276799

## Simulating the detector image

In [11]:
#Factor to consider SDD QE, MLI and filter absorption @ 8keV.
#           dead layer * QE    * 25 um Be * 300nm Al * 12.5um Kapton
qe_factor = 0.974      * 0.999 * 0.99527  * 0.99614  *  0.98945

eff_area = 0.01 * eff_area_vs_off_axis(mask.T.astype('int32'), bulk.T, ELXDIM, ELYDIM, FOCAL, MTHICK, ThetaX, ThetaY, degrees=True) * qe_factor
eff_area_onaxis = 0.01 * eff_area_vs_off_axis(mask.T.astype('int32'), bulk.T, ELXDIM, ELYDIM, FOCAL, MTHICK, 0, 0, degrees=True) * qe_factor

In [12]:
src_fluence = eff_area * Crab_on_axis_cm2 * exposure 
bkg_fluence = eff_area_onaxis * CXB_cm2 * exposure
print("Total source counts", src_fluence)
print("Total bkg counts", bkg_fluence)

Total source counts 167414.64088663555
Total bkg counts 488753.45573723695


In [13]:
ShiftX = -np.tan(np.deg2rad(ThetaX))*FOCAL/ELXDIM
ShiftY = -np.tan(np.deg2rad(ThetaY))*FOCAL/ELYDIM

In [14]:
datatype = "reconstructed"
detimage_src = model_source(ShiftX, ShiftY, src_fluence)
bkg_shape = solid_angle(bulk, ELXDIM, ELYDIM, FOCAL, nobulk=False)
detimage_bkg = bkg_shape/np.sum(bkg_shape) * bkg_fluence
detimage = detimage_src + detimage_bkg

## Reconstructing skyimage and varimage

In [15]:
sky = decode(detimage, rmatrix, bulk)
var = decode_var(detimage, rmatrix, bulk, FOCAL, ELXDIM, ELYDIM)

cx, cy = (sky.shape[0] - 1 ) // 2, (sky.shape[1] - 1 ) // 2

p_x = int(np.round(cx - ShiftX))
p_y = int(np.round(cy - ShiftY))

## Calculating SNR

In [16]:
print("Simulated source significance:", np.round(sky[p_x, p_y]//np.sqrt(var[p_x, p_y])))

Simulated source significance: 193.0
